<a href="https://colab.research.google.com/github/buiphamhaiha/crawl_poster_films/blob/main/crawl_poster_films_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Lấy dữ liệu tên film trong movie.dat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
!pip install chardet

In [ ]:
import chardet

In [ ]:
with open('/content/drive/MyDrive/website_film_python_project/movies.dat', 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']

data = pd.read_csv('/content/drive/MyDrive/website_film_python_project/movies.dat', delimiter='::', encoding=encoding)

<ipython-input-5-9f72c5877644>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('/content/drive/MyDrive/website_film_python_project/movies.dat', delimiter='::', encoding=encoding)


In [ ]:
#Chỉ lấy tên film và năm phát hành
film_id = data.iloc[:,0]
film_names = data.iloc[:, 1].str.split('(').str.get(0).tolist()
film_names = [name.rstrip() for name in film_names]

release_year = data.iloc[:, 1].str.split('(').str.get(1).str.split(')').str.get(0).tolist()
film_namess = []
for name in film_names:
    parts = name.split(", ")
    if len(parts) == 2:
    #nếu số từ sau dấu phẩy > số từ trước dấu phẩy thì giữ nguyên
    #nếu số từ sau dấu phẩy <= số từ trước dấu phẩy thì chuyển vị trí các từ sau dấu phẩy lên trước
        len_parts1 = len(parts[1].split())
        len_parts0 = len(parts[0].split())
        if len_parts1 <= len_parts0:
            film_name =  f'{parts[1]} {parts[0]}'
        else:
            film_name = name
        film_namess.append(film_name)
    else:
        film_namess.append(name)

#Tạo df

film_df = pd.DataFrame({
    'film_id': film_id,
    'film_namess': film_namess,
    'release_year': release_year
})

# Print the DataFrame
print(film_df)

film_df1 = film_df.iloc[-1500:]
print(film_df1)

film_df2 = film_df.iloc[0:1500]
print(film_df2)

film_df3 = film_df.iloc[1500:2382]
print(film_df3)

      film_id                  film_namess release_year
0           2                      Jumanji         1995
1           3             Grumpier Old Men         1995
2           4            Waiting to Exhale         1995
3           5  Father of the Bride Part II         1995
4           6                         Heat         1995
...       ...                          ...          ...
3877     3948             Meet the Parents         2000
3878     3949          Requiem for a Dream         2000
3879     3950                    Tigerland         2000
3880     3951             Two Family House         2000
3881     3952                The Contender         2000

[3882 rows x 3 columns]
      film_id               film_namess release_year
2382     2452  The Gate II: Trespassers         1990
2383     2453     The Boy Who Could Fly         1986
2384     2454                   The Fly         1958
2385     2455                   The Fly         1986
2386     2456                The Fly I

#2. Crawl dữ liệu poster film IMDB

In [ ]:
!pip install IMDbPY

In [ ]:
import imdb
import pandas as pd
import concurrent.futures

ia = imdb.IMDb()

In [ ]:
def get_exact_movie_info(movie_names, film_df1):
    # Tạo đối tượng IMDb
    ia = imdb.IMDb()

    exact_titles = []  # Danh sách các tên phim chính xác
    film_ids = []  # Danh sách các ID phim tương ứng
    film_release_years = []  # Danh sách các năm phát hành phim
    poster_urls = []  # Danh sách các URL ảnh bìa

    for i, movie_name in enumerate(movie_names):
        # Tìm kiếm phim bằng tên
        search_results = ia.search_movie(movie_name)

        exact_title = None  # Tên phim chính xác
        film_id = None  # ID phim
        film_release_year = None  # Năm phát hành phim
        poster_url = None  # URL ảnh bìa

        # Lấy các tên phim chính xác từ kết quả tìm kiếm
        if search_results:
            for result in search_results:
                if result['title'].lower() == movie_name.lower():
                    exact_title = result['title']
                    film_id = film_df1['film_id'].iloc[i]  # Lấy film_id từ df
                    film_release_year = film_df1['release_year'].iloc[i]  # Lấy release_year từ df
                    break
            if not exact_title:  # Nếu không có tên phim chính xác nào, chọn tên đầu tiên
                exact_title = search_results[0]['title']
                film_id = film_df1['film_id'].iloc[i]  # Lấy film_id từ df
                film_release_year = film_df1['release_year'].iloc[i]  # Lấy release_year từ df
        else:
            exact_title = "Không tìm thấy phim nào với tên này."

        # Lấy URL ảnh bìa
        if exact_title != "Không tìm thấy phim nào với tên này.":
            exact_movie = ia.get_movie(search_results[0].movieID)
            poster_url = exact_movie.get('cover url', 'Not available')

        exact_titles.append(exact_title)
        film_ids.append(film_id)
        film_release_years.append(film_release_year)
        poster_urls.append(poster_url)

    # Tạo DataFrame từ dữ liệu đã thu thập
    film_poster_df = pd.DataFrame({
        'film_id': film_ids,
        'film_name': exact_titles,
        'release_year': film_release_years,
        'poster_url': poster_urls
    })

    return film_poster_df

movie_names = film_df1['film_namess']
film_poster_df = get_exact_movie_info(movie_names, film_df1)

print(film_poster_df)

# Save DataFrame to CSV
# film_poster_df.to_csv('film_posters.csv', index=False)

      film_id                film_name release_year  \
0        2452  Gate 2: The Trespassers         1990   
1        2453    The Boy Who Could Fly         1986   
2        2454                  The Fly         1958   
3        2455                  The Fly         1986   
4        2456               The Fly II         1989   
...       ...                      ...          ...   
1495     3948         Meet the Parents         2000   
1496     3949      Requiem for a Dream         2000   
1497     3950                Tigerland         2000   
1498     3951         Two Family House         2000   
1499     3952            The Contender         2000   

                                             poster_url  
0     https://m.media-amazon.com/images/M/MV5BMGY1MT...  
1     https://m.media-amazon.com/images/M/MV5BMmIwYT...  
2     https://m.media-amazon.com/images/M/MV5BYTUzMm...  
3     https://m.media-amazon.com/images/M/MV5BYTUzMm...  
4     https://m.media-amazon.com/images/M/MV5BOTA

In [ ]:
def get_exact_movie_info(movie_names, film_df2):
    # Tạo đối tượng IMDb
    ia = imdb.IMDb()

    exact_titles = []  # Danh sách các tên phim chính xác
    film_ids = []  # Danh sách các ID phim tương ứng
    film_release_years = []  # Danh sách các năm phát hành phim
    poster_urls = []  # Danh sách các URL ảnh bìa

    for i, movie_name in enumerate(movie_names):
        # Tìm kiếm phim bằng tên
        search_results = ia.search_movie(movie_name)

        exact_title = None  # Tên phim chính xác
        film_id = None  # ID phim
        film_release_year = None  # Năm phát hành phim
        poster_url = None  # URL ảnh bìa

        # Lấy các tên phim chính xác từ kết quả tìm kiếm
        if search_results:
            for result in search_results:
                if result['title'].lower() == movie_name.lower():
                    exact_title = result['title']
                    film_id = film_df2['film_id'].iloc[i]  # Lấy film_id từ df
                    film_release_year = film_df2['release_year'].iloc[i]  # Lấy release_year từ df
                    break
            if not exact_title:  # Nếu không có tên phim chính xác nào, chọn tên đầu tiên
                exact_title = search_results[0]['title']
                film_id = film_df2['film_id'].iloc[i]  # Lấy film_id từ df
                film_release_year = film_df2['release_year'].iloc[i]  # Lấy release_year từ df
        else:
            exact_title = "Không tìm thấy phim nào với tên này."

        # Lấy URL ảnh bìa
        if exact_title != "Không tìm thấy phim nào với tên này.":
            exact_movie = ia.get_movie(search_results[0].movieID)
            poster_url = exact_movie.get('cover url', 'Not available')

        exact_titles.append(exact_title)
        film_ids.append(film_id)
        film_release_years.append(film_release_year)
        poster_urls.append(poster_url)

    # Tạo DataFrame từ dữ liệu đã thu thập
    film_poster_df1 = pd.DataFrame({
        'film_id': film_ids,
        'film_name': exact_titles,
        'release_year': film_release_years,
        'poster_url': poster_urls
    })

    return film_poster_df1

movie_names = film_df2['film_namess']
film_poster_df1 = get_exact_movie_info(movie_names, film_df2)

print(film_poster_df1)

# Save DataFrame to CSV
# film_poster_df.to_csv('film_posters.csv', index=False)

      film_id                               film_name     release_year  \
0           2                                 Jumanji             1995   
1           3                        Grumpier Old Men             1995   
2           4                       Waiting to Exhale             1995   
3           5             Father of the Bride Part II             1995   
4           6                                    Heat             1995   
...       ...                                     ...              ...   
1495     1533                             The Promise      La Promesse   
1496     1534                              Le Bonheur             1965   
1497     1535               Love! Valour! Compassion!             1997   
1498     1537                         Shall We Dance?  Shall We Dansu?   
1499     1538  The Second Jungle Book: Mowgli & Baloo             1997   

                                             poster_url  
0     https://m.media-amazon.com/images/M/MV5BZTk2Zm.

In [ ]:
film_poster_df.to_csv('film_posters.csv', index=False)
film_poster_df1.to_csv('film_posters1.csv', index=False)

In [ ]:
def get_exact_movie_info(movie_names, film_df3):
    # Tạo đối tượng IMDb
    ia = imdb.IMDb()

    exact_titles = []  # Danh sách các tên phim chính xác
    film_ids = []  # Danh sách các ID phim tương ứng
    film_release_years = []  # Danh sách các năm phát hành phim
    poster_urls = []  # Danh sách các URL ảnh bìa

    for i, movie_name in enumerate(movie_names):
        # Tìm kiếm phim bằng tên
        search_results = ia.search_movie(movie_name)

        exact_title = None  # Tên phim chính xác
        film_id = None  # ID phim
        film_release_year = None  # Năm phát hành phim
        poster_url = None  # URL ảnh bìa

        # Lấy các tên phim chính xác từ kết quả tìm kiếm
        if search_results:
            for result in search_results:
                if result['title'].lower() == movie_name.lower():
                    exact_title = result['title']
                    film_id = film_df3['film_id'].iloc[i]  # Lấy film_id từ df
                    film_release_year = film_df3['release_year'].iloc[i]  # Lấy release_year từ df
                    break
            if not exact_title:  # Nếu không có tên phim chính xác nào, chọn tên đầu tiên
                exact_title = search_results[0]['title']
                film_id = film_df3['film_id'].iloc[i]  # Lấy film_id từ df
                film_release_year = film_df3['release_year'].iloc[i]  # Lấy release_year từ df
        else:
            exact_title = "Không tìm thấy phim nào với tên này."

        # Lấy URL ảnh bìa
        if exact_title != "Không tìm thấy phim nào với tên này.":
            exact_movie = ia.get_movie(search_results[0].movieID)
            poster_url = exact_movie.get('cover url', 'Not available')

        exact_titles.append(exact_title)
        film_ids.append(film_id)
        film_release_years.append(film_release_year)
        poster_urls.append(poster_url)

    # Tạo DataFrame từ dữ liệu đã thu thập
    film_poster_df2 = pd.DataFrame({
        'film_id': film_ids,
        'film_name': exact_titles,
        'release_year': film_release_years,
        'poster_url': poster_urls
    })

    return film_poster_df2

movie_names = film_df3['film_namess']
film_poster_df2 = get_exact_movie_info(movie_names, film_df3)

print(film_poster_df2)

# Save DataFrame to CSV
film_poster_df2.to_csv('film_posters2.csv', index=False)

     film_id                      film_name release_year  \
0       1539                      Twin Town         1997   
1       1541               Addicted to Love         1997   
2       1542                    Brassed Off         1996   
3       1543         The Designated Mourner         1997   
4       1544  The Lost World: Jurassic Park         1997   
..       ...                            ...          ...   
877     2447                  Varsity Blues         1999   
878     2448                          Virus         1999   
879     2449    The Garbage Pail Kids Movie         1987   
880     2450                Howard the Duck         1986   
881     2451                       The Gate         1987   

                                            poster_url  
0    https://m.media-amazon.com/images/M/MV5BOTQ1MW...  
1    https://m.media-amazon.com/images/M/MV5BZDU5Zj...  
2    https://m.media-amazon.com/images/M/MV5BY2Y5ND...  
3    https://m.media-amazon.com/images/M/MV5BMTY2ND

In [ ]:
import imdb

def get_exact_movie_titles(movie_names):
    # Tạo đối tượng IMDb
    ia = imdb.IMDb()

    exact_titles = []  # Danh sách các tên phim chính xác

    for movie_name in movie_names:
        # Tìm kiếm phim bằng tên
        search_results = ia.search_movie(movie_name)

        exact_title = None  # Tên phim chính xác

        # Lấy các tên phim chính xác từ kết quả tìm kiếm
        if search_results:
            for result in search_results:
                if result['title'].lower() == movie_name.lower():
                    exact_title = result['title']
                    break
            if not exact_title:  # Nếu không có tên phim chính xác nào, chọn tên đầu tiên
                exact_title = search_results[0]['title']
        else:
            exact_title = "Không tìm thấy phim nào với tên này."

        exact_titles.append(exact_title)

    return exact_titles

# Danh sách các tên phim
movie_names = ['Slums of Beverly Hills. The Next Reel']
exact_titles = get_exact_movie_titles(movie_names)

ia = imdb.IMDb()
for exact_title in exact_titles:
    if exact_title != "Không tìm thấy phim nào với tên này.":
        # Lấy đúng vị trí của phim trong kết quả tìm kiếm
        movies = ia.search_movie(exact_title)
        exact_movie = None
        for movie in movies:
            if movie['title'].lower() == exact_title.lower():
                exact_movie = movie
                break

        if exact_movie:
            movie = ia.get_movie(exact_movie.movieID)
            poster_url = movie.get('cover url', 'Not available')
            print("Tên phim chính xác:", exact_title)
            # print("Năm phát hành:", exact_movie['year'])
            print("Link ảnh bìa:", poster_url)
            print()  # In ra một dòng trống giữa mỗi phim
        else:
            print("Không tìm thấy thông tin cho phim:", exact_title)
    else:
        print("Không tìm thấy phim nào với tên này.")


Tên phim chính xác: Slums of Beverly Hills. The Next Reel
Link ảnh bìa: Not available



In [ ]:
import imdb

ia = imdb.IMDb()
user_input = "Slums of Beverly Hills. The Next Reel"
moves = ia.search_movie(user_input)
poster_url = movies[0]['cover url']
print(poster_url)

https://m.media-amazon.com/images/M/MV5BODRjOGYwM2YtZWY1NS00Mzg5LTlkN2QtNGMxMDdiYTAxM2EwXkEyXkFqcGdeQXVyMTE3MTc3OTkz._V1_QL75_UY74_CR13,0,50,74_.jpg
